In [ ]:
# Instalando dependências básicas
!pip install pandas streamlit requests python-dotenv

# Instalando bibliotecas para o LLM Open Source (Usaremos Transformers para carregar um modelo)
!pip install transformers accelerate bitsandbytes sentencepiece


Passo A: No SEU COMPUTADOR (Onde a API está rodando)
Instale o ngrok:

Baixe o ngrok no site oficial e adicione-o ao PATH ou use-o na pasta onde ele foi baixado.

Crie uma conta gratuita no ngrok e obtenha seu Auth Token.

Exponha sua API (a porta 3000): Abra seu terminal ou prompt de comando (no seu computador) e execute:

ngrok http 3000

Anote o URL Público: O ngrok irá gerar um URL público temporário (ex: https://abcd-123-45-678-90.ngrok-free.app). Este é o seu novo URL_API.


In [ ]:
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from IPython.display import display, Markdown
import os
import requests
import pandas as pd

# ⚠️ SUBSTITUA ESTE VALOR PELO URL PÚBLICO GERADO PELO NGROK NO PASSO A3
NGROK_URL = "SUA_URL_NGROK_AQUI"
URL_API_REAL = f"{NGROK_URL}/api/transactions/dashboard"

# ⚠️ DEFINA SUA CHAVE DE API AQUI (Exemplo: se você a usa no .env)
# Se a chave for sensível, use Secrets do Colab, mas para simplificar, defina-a aqui.
API_DASHBOARD_KEY_VALUE = "SUA_CHAVE_API_AQUI"

# --- Função para Carregar Dados da API (Adaptada) ---
def carregar_dados_api_real():
    headers = {"x-api-key": API_DASHBOARD_KEY_VALUE}
    st.write(f"Tentando conectar em: {URL_API_REAL}")

    try:
        response = requests.get(URL_API_REAL, headers=headers, timeout=10) # Aumentei o timeout
        response.raise_for_status()

        data_json = response.json()
        df_bruto = pd.DataFrame(data_json.get("data", []))

        if df_bruto.empty:
            print("API retornou dados vazios.")
            return pd.DataFrame()

        # --- Flatten dos dados aninhados (Igual ao seu código original) ---
        if 'customer' in df_bruto.columns and 'product' in df_bruto.columns:
            df_bruto = pd.concat([
                df_bruto.drop(['customer', 'product'], axis=1),
                df_bruto['customer'].apply(pd.Series).add_prefix('customer_'),
                df_bruto['product'].apply(pd.Series).add_prefix('product_')
            ], axis=1)

        # --- Renomear e Mapear (Igual ao seu código original) ---
        df_renomeado = df_bruto.rename(columns={
            'totalPrice': 'preco_total_item',
            'date': 'data_pedido',
            'status': 'status_pedido',
            'customer_name': 'nome_cliente',
            'customer_segment': 'segmento_cliente',
            'customer_city': 'cidade',
            'customer_state': 'estado',
            'product_name': 'nome_produto',
            'product_quantity': 'quantidade'
        })

        # Mapeamento de região (copiado do seu código original)
        def map_region(estado):
            if not estado: return "Desconhecida"
            estado = estado.upper()
            if estado in ["AC", "AP", "AM", "PA", "RO", "RR", "TO"]: return "Norte"
            elif estado in ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"]: return "Nordeste"
            elif estado in ["DF", "GO", "MT", "MS"]: return "Centro-Oeste"
            elif estado in ["ES", "MG", "RJ", "SP"]: return "Sudeste"
            elif estado in ["PR", "RS", "SC"]: return "Sul"
            else: return "Desconhecida"

        df_renomeado['regiao'] = df_renomeado['estado'].apply(map_region)
        df_renomeado['data_pedido'] = pd.to_datetime(df_renomeado['data_pedido'])
        df_renomeado['mes_ano'] = df_renomeado['data_pedido'].dt.to_period('M').astype(str)

        return df_renomeado

    except requests.exceptions.RequestException as e:
        print(f"ERRO: Não foi possível conectar na API via ngrok. Garanta que o ngrok esteja rodando. Erro: {e}")
        return pd.DataFrame()

# --- Carrega os dados reais e calcula os KPIs ---
df = carregar_dados_api_real()

if df.empty:
    print("DataFrame vazio. Não foi possível rodar a análise.")
    # Evite que o código continue se os dados não carregarem
    raise SystemExit("Dados não carregados.")

# Criação do df_kpi para o prompt
receita_total = df['preco_total_item'].sum()
pedidos_unicos = df['orderId'].nunique()
df_kpi = pd.DataFrame([{
    'receita_total': receita_total,
    'pedidos_unicos': pedidos_unicos,
    'ticket_medio': receita_total / pedidos_unicos if pedidos_unicos > 0 else 0
}])

print("✅ Dados Reais Carregados com Sucesso!")
print(f"Total de Registros: {len(df)}")